In [1]:
import re
import pandas as pd
from bs4 import BeautifulSoup

### Konwertowanie pliku txt z <b>timestampami na </b> DataFrame

In [2]:
with open('Avengers_Age_of_Ultron(1).txt', 'r') as file:
    content = file.read()

soup = BeautifulSoup(content, 'html.parser')
content_notags = soup.get_text()
dialogue_lines = content_notags.split('\n\n')

starts, ends, lines = [], [], []
count = 0
last_symbol_was_comma = False

for i, line in enumerate(dialogue_lines):
    stripped_line = line.split('\n')[1:]
    timestamp = stripped_line[0]
    start = timestamp.split(' --> ')[0]
    end = timestamp.split(' --> ')[1]

    # sometimes 2 people talk at the same time

    punctuation_signs = {'.', '!', '?'}
    sentences = stripped_line[1:]
    if sentences[0][-1] not in punctuation_signs and len(sentences) > 1:
        sentences = [sentences[0] + ' ' + sentences[1]]

    for sentence in sentences:
        modified_sentence = ''.join(char for char in sentence if char != '"' and char != '-')

        if last_symbol_was_comma:
            # poprzednia linia tak naprawdę się nie zakończyła, więc chcemy ją dokleić do aktualnej
            last_index = len(lines)-1
            lines[last_index] += ' ' + modified_sentence.strip()
            # początku nie zmieniamy, ale zmieniamy koniec wypowiedzi
            ends[last_index] = end
            last_symbol_was_comma = False
        else:
            lines.append(modified_sentence.strip())
            starts.append(start)
            ends.append(end)

    if modified_sentence.endswith(','):
        count += 1 
        last_symbol_was_comma = True


print(count)
movie_captions = {
    'line': lines,
    'start': starts,
    'end': ends
}

df = pd.DataFrame(movie_captions)

89


In [3]:
print(df.head())
df.to_csv('with_timestamps.csv')

                                        line         start           end
0       Report to your stations immediately.  00:00:42,800  00:00:44,802
1  This is not a drill. We are under attack!  00:00:44,960  00:00:47,721
2                       We are under attack!  00:00:49,120  00:00:51,088
3                                      Shit!  00:01:52,200  00:01:54,487
4                                  Language!  00:01:52,200  00:01:54,487


### Konwertowanie pliku txt z <b>informacją, kto mówi</b> na DataFrame

In [4]:
with open('Avengers_Age_of_Ultron.txt', 'r') as file:
    content = file.read()

speakers = []
lines = []

def get_text(line: str):
    brackets_pattern = r'\[([^]]+)\]'
    text = re.sub(brackets_pattern, '', line)
    return text.strip()


for line in content.split('\n\n'):
    if ':' in line:
        speaker = line.split(':')[0]

        whole_text = get_text(line.split(':')[1])

        whole_text = whole_text.replace('...', '.').\
                                        replace('?', '.').\
                                             replace('!', '.').\
                                                replace('Dr.', 'Dr ')

        text_splitted = whole_text.split('.')[:-1]
        for sentence in text_splitted:
            modified_sentence = ''.join(char for char in sentence if char != '"' and char != '-')
            
            if line != '':
                lines.append(modified_sentence.strip())
            speakers.append(speaker)
    else:
        continue


movie_captions = {
    'speaker': speakers,
    'line': lines
}

df = pd.DataFrame(movie_captions)

In [5]:
print(df.line.head(20))

0                   Report to your stations immediately
1                                   This is not a drill
2                                   We are under attack
3                                   We are under attack
4                                                  Shit
5                                              Language
6                 JARVIS, what's the view from upstairs
7     The central building is protected by some kind...
8     Strucker's technology is well beyond any other...
9                           Loki's scepter must be here
10      Strucker couldn't mount this defense without it
11                                         At long last
12    â€śAt long lastâ€ť is lasting a little long, boys
13                                                 Yeah
14              I think we lost the element of surprise
15                                        Wait a second
16    No one else is going to deal with the fact tha...
17                                              

In [6]:
print(df.head())
df.to_csv('who_said_splitted.csv')

      speaker                                 line
0   Announcer  Report to your stations immediately
1   Announcer                  This is not a drill
2   Announcer                  We are under attack
3   Announcer                  We are under attack
4  Tony Stark                                 Shit
